In [ ]:
from langchain_ollama import ChatOllama
from langchain.chains import SequentialChain
from langchain import PromptTemplate, LLMChain
sentiment_prompt = PromptTemplate(input_variables=['text'],
                                  template='What is the sentiment (positive, neutral, negative) of the following text?')